# 🚀 Step 06: RAG 시스템 고도화

이 노트북에서는 기존 RAG 시스템을 개선하여 더 정확하고 유용한 답변을 제공하도록 만들어보겠습니다.
주요 개선 사항:
1. 프롬프트 엔지니어링
2. 컨텍스트 관리 개선
3. 답변 품질 향상

## 📚 1단계: 필요한 라이브러리 임포트

기존 라이브러리에 추가로 다음 기능들을 활용합니다:
- LangChain의 프롬프트 템플릿
- 답변 생성을 위한 고급 체인
- 컨텍스트 처리를 위한 유틸리티

In [ ]:
import os
import json
from typing import List, Dict
from dotenv import load_dotenv
import pandas as pd

# Elasticsearch 관련
from elasticsearch import Elasticsearch

# LangChain 관련
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory

# 환경 변수 로드
load_dotenv()

## 🎯 2단계: 개선된 프롬프트 템플릿 정의

더 정확하고 구조화된 답변을 위해 프롬프트 템플릿을 개선합니다.
이를 통해 시스템이 더 일관되고 유용한 답변을 제공할 수 있습니다.

In [ ]:
# 메타데이터 질의응답을 위한 프롬프트 템플릿
metadata_qa_template = """
당신은 데이터베이스 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 규칙을 따라주세요:
1. 테이블이나 컬럼명이 나오면 `백틱`으로 강조해주세요.
2. 데이터 타입이나 제약조건은 명확하게 설명해주세요.
3. 가능한 경우 예시 값을 포함해주세요.
4. 통계 정보가 있다면 함께 설명해주세요.

컨텍스트:
{context}

질문: {question}

답변: """

# 프롬프트 템플릿 생성
QA_PROMPT = PromptTemplate(
    template=metadata_qa_template,
    input_variables=["context", "question"]
)

## 🔄 3단계: 컨텍스트 관리 개선

검색된 문서의 관련성을 높이고, 대화 컨텍스트를 유지하여
더 자연스러운 대화가 가능하도록 개선합니다.

In [ ]:
# Elasticsearch 연결 설정
es = Elasticsearch(
    os.getenv('ELASTICSEARCH_URL'),
    basic_auth=(
        os.getenv('ELASTICSEARCH_USERNAME'),
        os.getenv('ELASTICSEARCH_PASSWORD')
    ),
    verify_certs=False
)

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# 개선된 벡터 저장소 설정
vector_store = ElasticsearchStore(
    es_connection=es,
    index_name="metadata-rag",
    embedding=embeddings
)

# 대화 기록을 저장할 메모리 초기화
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# ChatGPT 모델 초기화 (temperature 조정)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

# 개선된 QA 체인 생성
qa_chain = load_qa_chain(
    llm=llm,
    chain_type="stuff",
    prompt=QA_PROMPT,
    memory=memory
)

## 🔍 4단계: 개선된 검색 및 답변 생성

검색 결과의 품질을 높이고, 더 자연스러운 답변을 생성하는
함수를 구현합니다.

In [ ]:
def get_enhanced_answer(question: str, retriever=vector_store.as_retriever(search_kwargs={"k": 3})) -> str:
    """
    개선된 답변 생성 함수
    
    Args:
        question: 사용자 질문
        retriever: 문서 검색기
    
    Returns:
        str: 생성된 답변
    """
    # 관련 문서 검색
    docs = retriever.get_relevant_documents(question)
    
    # 문서가 없는 경우 처리
    if not docs:
        return "죄송합니다. 해당 질문에 대한 정보를 찾을 수 없습니다."
    
    # 컨텍스트 결합
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # QA 체인으로 답변 생성
    result = qa_chain({"input_documents": docs, "question": question})
    
    return result["output_text"]

# 테스트 질문 리스트
test_questions = [
    "customer_orders 테이블의 total_amount 컬럼에 대해 자세히 설명해주세요.",
    "주문 상태는 어떤 값들이 가능하고, NULL이 허용되나요?",
    "고객 정보는 어떤 테이블에 어떤 형태로 저장되나요?"
]

# 테스트 실행
print("🧪 개선된 RAG 시스템 테스트\n")
for question in test_questions:
    print(f"질문: {question}")
    print(f"답변: {get_enhanced_answer(question)}")
    print("-" * 80)

## 📊 5단계: 성능 평가

개선된 시스템의 성능을 평가하기 위한 지표들을 확인합니다.

In [ ]:
def evaluate_answer(question: str, answer: str) -> Dict[str, bool]:
    """
    답변 품질 평가 함수
    
    Args:
        question: 질문
        answer: 생성된 답변
    
    Returns:
        Dict[str, bool]: 평가 결과
    """
    return {
        "백틱 사용": "`" in answer,  # 테이블/컬럼명 강조
        "예시 포함": "[" in answer or "예:" in answer,  # 예시 값 포함
        "통계 정보": "최소" in answer or "최대" in answer or "개수" in answer,  # 통계 정보 포함
        "명확한 설명": len(answer.split()) >= 20  # 충분한 설명 길이
    }

# 평가 실행
print("📊 답변 품질 평가\n")
for question in test_questions:
    answer = get_enhanced_answer(question)
    results = evaluate_answer(question, answer)
    
    print(f"질문: {question}")
    for metric, passed in results.items():
        status = "✅" if passed else "❌"
        print(f"{status} {metric}")
    print("-" * 80)

## 🎯 주요 개선사항

1. 프롬프트 엔지니어링
   - 구조화된 프롬프트 템플릿
   - 일관된 포맷팅 규칙
   - 상세한 답변 가이드라인

2. 컨텍스트 관리
   - 대화 기록 유지
   - 관련 문서 검색 개선
   - 문서 결합 로직 개선

3. 답변 품질
   - 포맷팅 일관성
   - 예시 값 포함
   - 통계 정보 활용

4. 성능 평가
   - 객관적 평가 지표
   - 품질 모니터링
   - 개선 포인트 식별

이러한 개선을 통해 더 정확하고 유용한 답변을 제공할 수 있게 되었습니다!